In [21]:
from MultiLabelClassificationModel import MultiLabelClassificationModel, TOPICS, RANDOM_SEED

In [2]:
model = MultiLabelClassificationModel()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predic

In [17]:
import json 
import pandas as pd
import numpy as np
import re

LOCAL_PATH = "/Users/abraham/Competitive-programming-problems-classification/backend/"
DRIVE_PATH = "/content/drive/MyDrive/Competitive-programming-problems-classification/Github/Competitive-programming-problems-classification/backend"

PATH = LOCAL_PATH

problems = []
for topic in TOPICS:
  codeforcesProblems = json.load(open(f"{PATH}/data/codeforces-{topic}.json"))
  problems.extend(codeforcesProblems)
  # omegaupProblems = json.load(open(f"data/omegaup-{topic}.json"))
  # problems.extend(omegaupProblems)

data = []
for problem in problems:
    validTopics = [topic for topic in problem['topics'] if topic in TOPICS]
    text = problem['history']
    
    data.append({
      'text': text,
      'category': validTopics,
      'url': problem['url'],
    })

df = pd.DataFrame(data)
df.head()

,text,category,url
0,You are participating in Yet Another Tournamen...,"[greedy, sortings]",https://codeforces.com/problemset/problem/1783/C
1,An array a is called ugly if it contains at le...,"[math, sortings]",https://codeforces.com/problemset/problem/1783/A
2,Let' s call a string balanced if all character...,"[greedy, sortings, strings]",https://codeforces.com/problemset/problem/1781/C
3,A company of n people is planning a visit to t...,"[greedy, sortings]",https://codeforces.com/problemset/problem/1781/B
4,This is an interactive problem. Anya has gathe...,"[graphs, greedy, sortings]",https://codeforces.com/problemset/problem/1779/E


In [22]:
def getTopicCategory(topic):
  return f"category-{topic}"

In [23]:
for topic in TOPICS:
  newCategory = getTopicCategory(topic)
  df[newCategory] = df['category'].apply(lambda x: bool(topic in x))

df.head()

,text,category,url,category-sortings,category-strings,category-greedy,category-number theory,category-math,category-graphs,category-geometry,category-data structures,text-tokenized
0,You are participating in Yet Another Tournamen...,"[greedy, sortings]",https://codeforces.com/problemset/problem/1783/C,True,False,True,False,False,False,False,False,"[input_ids, attention_mask]"
1,An array a is called ugly if it contains at le...,"[math, sortings]",https://codeforces.com/problemset/problem/1783/A,True,False,False,False,True,False,False,False,"[input_ids, attention_mask]"
2,Let' s call a string balanced if all character...,"[greedy, sortings, strings]",https://codeforces.com/problemset/problem/1781/C,True,True,True,False,False,False,False,False,"[input_ids, attention_mask]"
3,A company of n people is planning a visit to t...,"[greedy, sortings]",https://codeforces.com/problemset/problem/1781/B,True,False,True,False,False,False,False,False,"[input_ids, attention_mask]"
4,This is an interactive problem. Anya has gathe...,"[graphs, greedy, sortings]",https://codeforces.com/problemset/problem/1779/E,True,False,True,False,False,True,False,False,"[input_ids, attention_mask]"


In [19]:
# Do binary classification to see which categories are confusing

from nlpBert import tokenize

df['text-tokenized'] = df['text'].apply(lambda x : tokenize(x))
df.head()

,text,category,url,category-sortings,category-strings,category-greedy,category-number theory,category-math,category-graphs,category-geometry,category-data structures,text-tokenized
0,You are participating in Yet Another Tournamen...,"[greedy, sortings]",https://codeforces.com/problemset/problem/1783/C,True,False,True,False,False,False,False,False,"[input_ids, attention_mask]"
1,An array a is called ugly if it contains at le...,"[math, sortings]",https://codeforces.com/problemset/problem/1783/A,True,False,False,False,True,False,False,False,"[input_ids, attention_mask]"
2,Let' s call a string balanced if all character...,"[greedy, sortings, strings]",https://codeforces.com/problemset/problem/1781/C,True,True,True,False,False,False,False,False,"[input_ids, attention_mask]"
3,A company of n people is planning a visit to t...,"[greedy, sortings]",https://codeforces.com/problemset/problem/1781/B,True,False,True,False,False,False,False,False,"[input_ids, attention_mask]"
4,This is an interactive problem. Anya has gathe...,"[graphs, greedy, sortings]",https://codeforces.com/problemset/problem/1779/E,True,False,True,False,False,True,False,False,"[input_ids, attention_mask]"


In [20]:
from sklearn.model_selection import train_test_split


for topic in TOPICS:
  newCategory = getTopicCategory(topic)
  x_train, x_test, y_train, y_test = train_test_split(df['text-tokenized'], df[newCategory], test_size=0.1, random_state=RANDOM_SEED, shuffle=True)


{'input_ids': tensor([[  101,  1192,  1132,  7828,  1107,  6355,  2543,  5629,   119,  1247,
          1132,   183,   116,   122,  6635,   131,  1128,  1105,   183,  1168,
          7741,   117,  8324,  1121,   122,  1106,   183,   119,  2994,  1160,
          6635,  1209,  1505,  1222,  1296,  1168,  2839,  1517,   119,  1409,
          1103,  6566,   178,  2399,  1222,  1103,  6566,   179,   117,  1119,
          4646,  1191,  1105,  1178,  1191,   178,   135,   179,   119,  1332,
          1103,  6566,   178,  2399,  1222,  1128,   117,  1917,  3316,   170,
          1376,  2113,  8277,   119,  1130,  1546,  1106,  1243,   170,  1782,
          1222,  6566,   178,   117,  1128,  1444,  1106,  7034,  1111,  1103,
          1801,  1111,  1120,  1655,   170,   168,   178,  1904,   783,  4303,
           117,  1128,  3857,  1106,  1115,  6566,   119,  1192,  1138,   182,
          1904,  1107,  1703,  1106,  7034,  1111,  2697,   117,  1133,  1128,
          1169,  7034,  1111,  1178,  